# AZML III Pipeline - hyper parametre et Auto ml
Laurent Cetinsoy


## AutoML

Le machine learning, c'est bien. Le terme buzz du moment c'est AutoML : automatiser la recherche du meilleur modèle ou des meilleurs hyper paramètres pour un dataset donné. Plein d'entreprises proposent des outils pour le faire et microsoft ne déroge pas à la règle : le sdk permet d'en faire.  

Reférence utile : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-configure-auto-train  
https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning

Les concepts / classes utiles sont : AutoMLConfig

`automl_config = AutoMLConfig(…)
automl_run = experiment.submit(automl_config …)
best_run, fitted_model = automl_run.get_output()`

Charger le Workspace comme d'hab

In [1]:
from azureml.core import Workspace
ws=Workspace.create(name ="aro-lctp4",
                     subscription_id="",
                     resource_group="",
                   location='eastus2')
print(azureml.core.VERSION,ws.name)

Deploying AppInsights with name arolctp4insights1e63ea64.
Deployed AppInsights with name arolctp4insights1e63ea64. Took 3.43 seconds.
Deploying KeyVault with name arolctp4keyvaultcbbd3fdb.
Deploying StorageAccount with name arolctp4storage312e4f10f.
Deployed KeyVault with name arolctp4keyvaultcbbd3fdb. Took 18.99 seconds.
Deploying Workspace with name aro-lctp4.
Deployed StorageAccount with name arolctp4storage312e4f10f. Took 21.05 seconds.
Deployed Workspace with name aro-lctp4. Took 20.2 seconds.
1.5.0 aro-lctp4


In [2]:
from azureml.core import Workspace
ws=Workspace.from_config()
print(azureml.core.VERSION,ws.name)

WorkspaceException: WorkspaceException:
	Message: No workspaces found with name=aro-lctp3 in subscription=68bdd703-8837-469c-80bd-bfb35f3b886f
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "No workspaces found with name=aro-lctp3 in subscription=68bdd703-8837-469c-80bd-bfb35f3b886f"
    }
}

### Regression

On va entraîner un modèle de régression sur le dataset house. Charger le dataset house.csv avec pandas

In [ ]:
import pandas as pd
df = pd.read_csv("houses_automl.csv")

In [ ]:
df.head(5)

Faire un train test split

In [ ]:
from sklearn.model_selection import train_test_split
#X= df.drop(['price'], axis=1)
#y= df['price']

X_train, X_test = train_test_split(df, test_size=0.1, random_state=42)
label = "price"
X_train.head(5)

Créer une experience "myautoml_classification"

In [ ]:
from azureml.train.automl import AutoMLConfig
# task can be one of classification, regression, forecasting
automl_config = AutoMLConfig(task = "regression")


Créer un objet de la classe AutoMLConfig. Veiller à limiter la durée de l'auto ML à 5 minutes maximum

In [ ]:
automl_regressor = AutoMLConfig(
           task='regression',
           experiment_timeout_minutes=15,
           whitelist_models=['SGD'],
           primary_metric='normalized_mean_absolute_error',
           training_data=X_train,
           label_column_name=label,
           n_cross_validations=5)

Lancer votre experience avec le submit sur l'objet de config

In [ ]:
from azureml.core.experiment import Experiment

experiment_name = 'automl-regression'
project_folder = './sample_projects/automl-regression'

experiment = Experiment(ws, experiment_name)

In [ ]:
automl_run = experiment.submit(automl_regressor, show_output=True)

Récupérer le meilleur modèle

In [ ]:
best_run, fitted_model = automl_run.get_output()

In [ ]:
best_run

In [ ]:
print(type(fitted_model))
fitted_model

In [ ]:
#prediction
y_pred = fitted_model.predict(X_test.drop(['price'],axis=1))
y_pred

### Bonus (si le temps ) faire la même chose avec un dataset de classification de votre choix comme titanic


## Hyper drive 

Kezako ? Module d'azure machine learning qui permet de faire de la recherche d'hyper parametres.

Reférences utiles : 
- https://docs.microsoft.com/fr-fr/python/api/azureml-train-core/azureml.train.hyperdrive?view=azure-ml-py
- https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

Les classes concepts intéressant sont HyperParameterSampling, HyperDriveRun

Quelle différence entre hyperdrive et automl ? C'est pas la même chose ? Pourquoi deux choses différentes ? 

A quoi sert la classe HyperParameterSampling ? 


`Classe de base abstraite pour tous les algorithmes d’échantillonnage hyperparamétrique.
Cette classe encapsule l’espace hyperparamétrique, la méthode d’échantillonnage et les propriétés supplémentaires pour les classes d’échantillonnage dérivées : BayesianParameterSampling, GridParameterSamplinget RandomParameterSampling.`

Quelles sont les principales classes filles ? 

`BayesianParameterSampling, GridParameterSamplinget RandomParameterSampling`

Quelles semblent-être les méthodes les plus importantes ? 

`HyperParameterSampling(sampling_method_name, 
                        parameter_space, 
                        properties=None, 
                        supported_distributions=None, 
                        distributions_validators=None)`

`to_json() 	
Retourne le JSON de l’objet d’échantillonnage hyperparamétrique.`

A quoi sert la classe HyperDriveRun ? 

`HyperDriveRun contient les détails d’une expérience Hyperdrive soumise.
Cette classe peut être utilisée pour gérer, vérifier l’État et récupérer les détails de la série de tests pour l’exécution de l’hyperlecteur et pour chaque exécution enfant générée.`

On va faire la recherche d'hyperparamètre sur notre modèle de régression de tout à l'heure (house).
Créer un dictionnaire avec les noms des paramètres que vous voulez recherche (par exemple le learning rate ou le taux de régularisation), cf https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#define-search-space

Instancier un objet de la classe RandomParameterSampling

Créer un estimateur et soumettre l'experience localement

In [3]:
%%writefile train2.py
# script

import pandas as pd
import numpy as np
import argparse
from azureml.core import Run
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error

parser = argparse.ArgumentParser()
parser.add_argument('--eta0', type=float, default=0.01, dest = 'eta0')
parser.add_argument('--learning_rate', type=str,default='constant' ,dest='learning_rate')
args = parser.parse_args()
eta0  = args.eta0
learning_rate = args.learning_rate

run = Run.get_context()

#data

df = pd.read_csv("houses_automl.csv")
X= df.drop(['price'], axis=1)
X= X.drop(['orientation'], axis=1)
y= df['price']
X_train, X_test,y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42)

#model
model = SGDRegressor(eta0=eta0, learning_rate=learning_rate)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#log
run.log("mean_absolute_error", mean_absolute_error(y_test, y_pred))

run.complete()
   


Overwriting train2.py


In [45]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.core import Experiment

script_params = {
                 '--eta0': 0.01  
                }
estimator = SKLearn('./', 
                    script_params=script_params,
                    compute_target="local",
                    entry_script="./train2.py")

experiment = Experiment(ws,'experiment-estimator')
run = experiment.submit(config = estimator)
RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'experiment-estimator_1589482410_1dc23020',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-05-14T18:53:33.139551Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '5dd8931d-0d13-4403-a6c5-40f4a95ce27e'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train2.py',
  'useAbsolutePath': False,
  'arguments': ['--eta0', '0.01'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment experiment-estimator Environment',
   'version': 'Autosave_2020-05-14T18:31:39Z_cbcae952',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'condaDependencies': {'channels': ['anaconda', 'conda-forge'],
     'dependencies': ['python=3.6.2',
      {'pip': ['azureml-defaults',
        'scikit-learn==0.20.3',
        '

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_cluster_name = "cmp-cluster"

compute_config = AmlCompute.provisioning_configuration(vm_size ="STANDARD_D2_V2",
                                                        max_nodes=4) 
compute_cluster = ComputeTarget.create(workspace = ws,
                                      name= compute_cluster_name,
                                      provisioning_configuration = compute_config)

compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.train.hyperdrive import *

param_sampling = RandomParameterSampling({
                            "--eta0": choice(0.01,0.1),
                            "--learning_rate": choice('constant','invscaling')
                            })

In [6]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails
from azureml.core import Experiment

script_params = {
                 '--eta0': 0.01,
                 '--learning_rate':'constant'   
                }
estimator = SKLearn('./', 
                    script_params=script_params,
                    compute_target=compute_cluster,
                    entry_script="./train2.py")

In [7]:
from azureml.train.hyperdrive import HyperDriveConfig

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                          hyperparameter_sampling=param_sampling, 
                          policy=None,
                          primary_metric_name="mean_absolute_error",
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                          max_total_runs=5,
                          max_concurrent_runs=4)

In [8]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, 'experiment_hyperdrive')
hyperdrive_run = experiment.submit(hyperdrive_run_config)
RunDetails(hyperdrive_run).show()

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--eta0', '--learning_rate'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('Best Run Id: ', best_run.id)
best_run_metrics = best_run.get_metrics()
best_run_metrics

Best Run Id:  HD_0c3d92d9-6be6-45f1-867d-50bc775349ce_0


{'mean_absolute_error': 24972648581766.91}